# Notebook 04: Entrenamiento y Comparación de Modelos

Este notebook entrena y compara múltiples arquitecturas de CNN para reconocimiento de señas ASL:

**Modelos a entrenar:**
1. SimpleCNN (Baseline)
2. ImprovedCNN (Con BatchNormalization)
3. DeepCNN (Arquitectura más profunda)

**Opcionalmente (si hay suficiente memoria):**
4. EfficientNetB0 (Transfer Learning)
5. MobileNetV2 (Transfer Learning - Ligero)
6. ResNet50 (Transfer Learning - Residual)

## 1. Configuración del Entorno

In [7]:
import os
import sys
from pathlib import Path

# Detectar entorno
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    from google.colab import drive
    print("Ejecutando en Google Colab. Montando Drive...")
    drive.mount('/content/drive')
    
    # RUTA DEL PROYECTO EN DRIVE
    PROJECT_ROOT = Path('/content/drive/MyDrive/proyecto-app-lenguaje-senas')
    
    if not PROJECT_ROOT.exists():
        print(f"Carpeta no encontrada en {PROJECT_ROOT}!")
    else:
        os.chdir(PROJECT_ROOT)
        if str(PROJECT_ROOT) not in sys.path:
            sys.path.append(str(PROJECT_ROOT))
        print(f"Entorno Colab configurado. Directorio: {os.getcwd()}")
else:
    # Entorno Local
    PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
    os.chdir(PROJECT_ROOT)
    if str(PROJECT_ROOT) not in sys.path:
        sys.path.append(str(PROJECT_ROOT))
    print(f"Entorno Local configurado. Directorio: {os.getcwd()}")

# Importaciones básicas
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

print(f"TensorFlow version: {tf.__version__}")
if tf.config.list_physical_devices('GPU'):
    print("GPU disponible")
else:
    print("GPU no disponible. El entrenamiento será LENTO.")

Ejecutando en Google Colab. Montando Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Entorno Colab configurado. Directorio: /content/drive/MyDrive/proyecto-app-lenguaje-senas
TensorFlow version: 2.19.0
GPU disponible


### 1.1. Instalación de Dependencias (Solo en Colab)

In [8]:
if IN_COLAB:
    print("Instalando dependencias...")
    !pip install -q kaggle tensorflow opencv-python scikit-learn seaborn plotly
    print("Dependencias instaladas correctamente")

Instalando dependencias...
Dependencias instaladas correctamente


### 1.2. Configuración de Rutas y Estructura

In [9]:
# Definir directorios de trabajo basados en la raíz del proyecto
DIRS = {
    'data_raw': PROJECT_ROOT / 'data' / 'raw',
    'data_processed': PROJECT_ROOT / 'data' / 'processed',
    'models': PROJECT_ROOT / 'models',
    'results': PROJECT_ROOT / 'results',
    'results_figures': PROJECT_ROOT / 'results' / 'figures',
    'results_reports': PROJECT_ROOT / 'results' / 'reports',
}

# Asegurar que existan todos los directorios
for dir_name, dir_path in DIRS.items():
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✓ {dir_name}: {dir_path}")


✓ data_raw: /content/drive/MyDrive/proyecto-app-lenguaje-senas/data/raw
✓ data_processed: /content/drive/MyDrive/proyecto-app-lenguaje-senas/data/processed
✓ models: /content/drive/MyDrive/proyecto-app-lenguaje-senas/models
✓ results: /content/drive/MyDrive/proyecto-app-lenguaje-senas/results
✓ results_figures: /content/drive/MyDrive/proyecto-app-lenguaje-senas/results/figures
✓ results_reports: /content/drive/MyDrive/proyecto-app-lenguaje-senas/results/reports


### 1.3. Configuración de Kaggle (si es necesario descargar datos)

In [10]:
if IN_COLAB:
    from google.colab import files
    import json
    
    # Verificar si ya tenemos datos
    if not (DIRS['data_raw'] / 'sign_mnist_train.csv').exists():
        print("Datos no encontrados. Subir kaggle.json para descargar...")
        print("O ejecutar notebook 01_descarga_datos.ipynb primero")
        
        # Opcional: subir kaggle.json
        # uploaded = files.upload()
        # if 'kaggle.json' in uploaded:
        #     !mkdir -p ~/.kaggle
        #     !mv kaggle.json ~/.kaggle/
        #     !chmod 600 ~/.kaggle/kaggle.json
    else:
        print("Datos encontrados")

Datos encontrados


## 2. Importar Librerías

In [11]:
# Configuración de warnings
import warnings
warnings.filterwarnings('ignore')

# Core
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

# Métricas y utilidades
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, 
    confusion_matrix,
    accuracy_score,
    precision_recall_fscore_support
)

# Utilidades
import json
import pickle
from datetime import datetime
from tqdm.auto import tqdm
import time

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Configuración de TensorFlow
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")

# Verificar GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f"\n✓ GPU disponible: {len(gpus)} dispositivo(s)")
    for gpu in gpus:
        print(f"  - {gpu}")
        tf.config.experimental.set_memory_growth(gpu, True)
else:
    print("\n⚠ No se detectó GPU, usando CPU")

# Semillas para reproducibilidad
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("\n✓ Librerías importadas correctamente")

TensorFlow version: 2.19.0
Keras version: 3.10.0

✓ GPU disponible: 1 dispositivo(s)
  - PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')

✓ Librerías importadas correctamente


## 3. Cargar Datos Preprocesados

Cargamos los datos que fueron preprocesados en el notebook 03.

In [15]:
# Verificar archivos preprocesados
print("Buscando datos preprocesados...")

# Opción 1: Archivos .npz (comprimidos - del notebook 03)
data_standard_npz = DIRS['data_processed'] / 'data_standard.npz'
data_tl_npz = DIRS['data_processed'] / 'data_transfer_learning_96.npz'

# Opción 2: Archivos .npy individuales
processed_files_npy = [
    'X_train.npy',
    'X_val.npy', 
    'X_test.npy',
    'y_train.npy',
    'y_val.npy',
    'y_test.npy'
]

all_npy_exist = all(
    (DIRS['data_processed'] / f).exists() 
    for f in processed_files_npy
)

# Prioridad: .npz (comprimido) > .npy individual > raw
if data_standard_npz.exists():
    print("✓ Encontrado: data_standard.npz (desde notebook 03)")
    data = np.load(data_standard_npz)
    X_train = data['X_train']
    X_val = data['X_val']
    X_test = data['X_test']
    y_train = data['y_train']
    y_val = data['y_val']
    y_test = data['y_test']
    print("✓ Datos preprocesados cargados desde .npz")
    
elif all_npy_exist:
    print("✓ Encontrados archivos .npy individuales")
    X_train = np.load(DIRS['data_processed'] / 'X_train.npy')
    X_val = np.load(DIRS['data_processed'] / 'X_val.npy')
    X_test = np.load(DIRS['data_processed'] / 'X_test.npy')
    y_train = np.load(DIRS['data_processed'] / 'y_train.npy')
    y_val = np.load(DIRS['data_processed'] / 'y_val.npy')
    y_test = np.load(DIRS['data_processed'] / 'y_test.npy')
    print("✓ Datos preprocesados cargados desde .npy")
    
else:
    print("⚠ Archivos preprocesados no encontrados.")
    print(f"   Buscaba en: {DIRS['data_processed']}")
    print("\nIntentando cargar datos raw como alternativa...")
    
    # Cargar datos raw
    train_df = pd.read_csv(DIRS['data_raw'] / 'sign_mnist_train.csv')
    test_df = pd.read_csv(DIRS['data_raw'] / 'sign_mnist_test.csv')
    
    # Separar features y labels
    X_train_raw = train_df.drop('label', axis=1).values
    y_train_raw = train_df['label'].values
    X_test = test_df.drop('label', axis=1).values
    y_test = test_df['label'].values
    
    # Reshape y normalizar
    X_train_raw = X_train_raw.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    X_test = X_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0
    
    # Split train/val
    X_train, X_val, y_train, y_val = train_test_split(
        X_train_raw, y_train_raw, 
        test_size=0.15, 
        random_state=SEED,
        stratify=y_train_raw
    )
    
    print(f"✓ Datos cargados desde raw")

# Información de los datos
print(f"\nForma de los datos:")
print(f"  X_train: {X_train.shape}")
print(f"  X_val: {X_val.shape}")
print(f"  X_test: {X_test.shape}")
print(f"  y_train: {y_train.shape}")
print(f"  y_val: {y_val.shape}")
print(f"  y_test: {y_test.shape}")

# Detectar si y_train ya está en one-hot encoding
is_one_hot_encoded = (len(y_train.shape) == 2) and (y_train.ndim == 2)

if is_one_hot_encoded:
    # Ya está en one-hot (forma: N, num_classes)
    NUM_CLASSES = y_train.shape[1]
    y_train_cat = y_train
    y_val_cat = y_val
    y_test_cat = y_test
    print(f"\nDatos detectados en ONE-HOT ENCODING")
else:
    # Está en formato de etiquetas (0, 1, 2, ...)
    NUM_CLASSES = len(np.unique(y_train))
    y_train_cat = to_categorical(y_train, NUM_CLASSES)
    y_val_cat = to_categorical(y_val, NUM_CLASSES)
    y_test_cat = to_categorical(y_test, NUM_CLASSES)
    print(f"\nConvirtiendo etiquetas a ONE-HOT ENCODING")

print(f"Número de clases: {NUM_CLASSES}")
print(f"Forma y_train_cat: {y_train_cat.shape}")

print("\nDatos listos para entrenamiento")

Buscando datos preprocesados...
✓ Encontrado: data_standard.npz (desde notebook 03)
✓ Datos preprocesados cargados desde .npz

Forma de los datos:
  X_train: (21964, 28, 28, 1)
  X_val: (5491, 28, 28, 1)
  X_test: (7172, 28, 28, 1)
  y_train: (21964, 25)
  y_val: (5491, 25)
  y_test: (7172, 25)

Datos detectados en ONE-HOT ENCODING
Número de clases: 25
Forma y_train_cat: (21964, 25)

Datos listos para entrenamiento


## 4. Definir Arquitecturas de Modelos

Definimos todas las arquitecturas que vamos a entrenar y comparar.

In [17]:
def create_simple_cnn(input_shape=(28, 28, 1), num_classes=24):
    """
    CNN Simple - Baseline
    
    Arquitectura básica con 2 bloques convolucionales.
    """
    model = models.Sequential([
        # Bloque 1
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 2
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Capas densas
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ], name='SimpleCNN')
    
    return model


def create_improved_cnn(input_shape=(28, 28, 1), num_classes=24):
    """
    CNN Mejorada con BatchNormalization
    
    Incluye normalización por lotes para acelerar convergencia.
    """
    model = models.Sequential([
        # Bloque 1
        layers.Conv2D(32, (3, 3), padding='same', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(32, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 2
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.Conv2D(64, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 3
        layers.Conv2D(128, (3, 3), padding='same'),
        layers.BatchNormalization(),
        layers.Activation('relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Capas densas
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ], name='ImprovedCNN')
    
    return model


def create_deep_cnn(input_shape=(28, 28, 1), num_classes=24):
    """
    CNN Profunda con Skip Connections
    
    Arquitectura más profunda inspirada en ResNet.
    """
    inputs = layers.Input(shape=input_shape)
    
    # Bloque inicial
    x = layers.Conv2D(32, (3, 3), padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    
    # Bloque residual 1
    residual = x
    x = layers.Conv2D(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(32, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Bloque residual 2
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    residual = x
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Add()([x, residual])
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Dropout(0.25)(x)
    
    # Capas finales
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name='DeepCNN')
    return model


def create_transfer_learning_model(base_model_name='EfficientNetB0', 
                                   input_shape=(28, 28, 1), 
                                   num_classes=24,
                                   trainable=False):
    """
    Modelo usando Transfer Learning
    
    Args:
        base_model_name: Nombre del modelo base
        input_shape: Forma de entrada
        num_classes: Número de clases
        trainable: Si las capas del modelo base son entrenables
    """
    # Adaptar entrada para modelos preentrenados
    inputs = layers.Input(shape=input_shape)
    
    # Convertir grayscale a RGB (los modelos preentrenados esperan 3 canales)
    x = layers.Conv2D(3, (1, 1), padding='same')(inputs)
    
    # Redimensionar a tamaño mínimo requerido
    if base_model_name in ['EfficientNetB0', 'MobileNetV2']:
        target_size = 32  # Tamaño mínimo aceptable
    else:
        target_size = 32
    
    x = layers.Resizing(target_size, target_size)(x)
    
    # Cargar modelo base
    if base_model_name == 'EfficientNetB0':
        base_model = tf.keras.applications.EfficientNetB0(
            include_top=False,
            weights='imagenet',
            input_shape=(target_size, target_size, 3),
            pooling='avg'
        )
    elif base_model_name == 'MobileNetV2':
        base_model = tf.keras.applications.MobileNetV2(
            include_top=False,
            weights='imagenet',
            input_shape=(target_size, target_size, 3),
            pooling='avg'
        )
    elif base_model_name == 'ResNet50':
        base_model = tf.keras.applications.ResNet50(
            include_top=False,
            weights='imagenet',
            input_shape=(target_size, target_size, 3),
            pooling='avg'
        )
    else:
        raise ValueError(f"Modelo base no reconocido: {base_model_name}")
    
    base_model.trainable = trainable
    
    # Construir modelo completo
    x = base_model(x, training=False)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    
    model = models.Model(inputs=inputs, outputs=outputs, name=f'TL_{base_model_name}')
    return model


print("Arquitecturas definidas")

Arquitecturas definidas


## 5. Configuración de Entrenamiento

Definimos callbacks, data augmentation y parámetros de entrenamiento.

In [32]:
# Parámetros de entrenamiento
if IN_COLAB:
    BATCH_SIZE = 64
    EPOCHS = 30
    LEARNING_RATE = 0.001
    EARLY_STOPPING_PATIENCE = 5
    TRAIN_ALL_MODELS = True
else:
    BATCH_SIZE = 128
    EPOCHS = 50
    LEARNING_RATE = 0.001
    EARLY_STOPPING_PATIENCE = 10
    TRAIN_ALL_MODELS = True

print(f"Configuración de entrenamiento (Entorno: {'Colab' if IN_COLAB else 'Local'})")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Épocas máx: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")
print(f"  Early stopping patience: {EARLY_STOPPING_PATIENCE}")
print(f"  Entrenar todos modelos: {TRAIN_ALL_MODELS}")

Configuración de entrenamiento (Entorno: Colab)
  Batch size: 64
  Épocas máx: 30
  Learning rate: 0.001
  Early stopping patience: 5
  Entrenar todos modelos: True


In [19]:
def create_callbacks(model_name):
    """
    Crea callbacks para el entrenamiento
    
    Args:
        model_name: Nombre del modelo
    
    Returns:
        Lista de callbacks
    """
    model_path = DIRS['models'] / f"{model_name}.keras"
    
    callbacks_list = [
        # Guardar mejor modelo
        callbacks.ModelCheckpoint(
            str(model_path),
            monitor='val_accuracy',
            save_best_only=True,
            mode='max',
            verbose=1
        ),
        
        # Early stopping
        callbacks.EarlyStopping(
            monitor='val_loss',
            patience=EARLY_STOPPING_PATIENCE,
            restore_best_weights=True,
            verbose=1,
            min_delta=0.001  # Variación mínima para considerarlo mejora
        ),
        
        # Reducir learning rate
        callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7,
            verbose=1
        ),
    ]
    
    return callbacks_list


# Data augmentation
if IN_COLAB:
    # Augmentation más ligero
    train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.08,
        height_shift_range=0.08,
        zoom_range=0.1,
        shear_range=0.05,
        fill_mode='nearest'
    )
else:
    # Augmentation completo para máquina local
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.15,
        brightness_range=[0.5, 1.5],
        shear_range=0.1,
        fill_mode='nearest'
    )

# Generador de validación (sin augmentation)
val_datagen = ImageDataGenerator()

print(f"\nCallbacks y data augmentation configurados ({'ligero' if IN_COLAB else 'completo'})")


Callbacks y data augmentation configurados (ligero)


## 6. Función de Entrenamiento

In [21]:
def train_model(model, model_name, X_train, y_train, X_val, y_val, 
                epochs=EPOCHS, batch_size=BATCH_SIZE, use_augmentation=True):
    """
    Entrena un modelo y guarda resultados
    
    Args:
        model: Modelo de Keras
        model_name: Nombre del modelo
        X_train, y_train: Datos de entrenamiento
        X_val, y_val: Datos de validación
        epochs: Número de épocas
        batch_size: Tamaño de batch
        use_augmentation: Usar data augmentation
    
    Returns:
        history: Historial de entrenamiento
    """
    print(f"\n{'='*60}")
    print(f"Entrenando: {model_name}")
    print(f"{'='*60}")
    
    # Compilar modelo
    if IN_COLAB:
        # Mixed precision training
        try:
            from tensorflow.keras import mixed_precision
            policy = mixed_precision.Policy('mixed_float16')
            mixed_precision.set_global_policy(policy)
        except:
            pass
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=LEARNING_RATE),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    # Mostrar resumen
    total_params = model.count_params()
    print(f"\nParámetros totales: {total_params:,}")
    print(f"Tamaño estimado del modelo: {(total_params * 4) / (1024**2):.1f} MB")
    
    # Crear callbacks
    callbacks_list = create_callbacks(model_name)
    
    # Entrenar
    start_time = time.time()
    
    if use_augmentation:
        # Con data augmentation
        train_generator = train_datagen.flow(
            X_train, y_train, 
            batch_size=batch_size,
            shuffle=True
        )
        
        history = model.fit(
            train_generator,
            steps_per_epoch=len(X_train) // batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks_list,
            verbose=1
        )
    else:
        # Sin data augmentation
        history = model.fit(
            X_train, y_train,
            batch_size=batch_size,
            epochs=epochs,
            validation_data=(X_val, y_val),
            callbacks=callbacks_list,
            verbose=1
        )
    
    training_time = time.time() - start_time
    
    # Limpiar memoria
    import gc
    gc.collect()
    
    print(f"\n✓ Entrenamiento completado en {training_time/60:.2f} minutos")
    print(f"  Épocas completadas: {len(history.history['loss'])}")
    print(f"  Best validation accuracy: {max(history.history.get('val_accuracy', [0])):.4f}")
    
    # Guardar historial
    history_path = DIRS['results_reports'] / f'{model_name}_history.json'
    with open(history_path, 'w') as f:
        json.dump({
            'history': str(history.history),
            'training_time': training_time,
            'epochs_trained': len(history.history['loss']),
            'best_val_accuracy': float(max(history.history.get('val_accuracy', [0])))
        }, f)
    
    return history


print("Función de entrenamiento definida")

Función de entrenamiento definida


## 7. Entrenar Modelos

Entrenamos todos los modelos definidos.

### 7.1. SimpleCNN (Baseline)

In [23]:
# Crear y entrenar SimpleCNN
model_simple = create_simple_cnn(num_classes=NUM_CLASSES)
model_simple.summary()

history_simple = train_model(
    model_simple, 
    'SimpleCNN',
    X_train, y_train_cat,
    X_val, y_val_cat
)

Model: "SimpleCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 5, 5, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 1600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       204,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 25)             │         3,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 226,969 (886.60 KB)

 Trainable params: 226,969 (886.60 KB)

 Non-trainable params: 0 (0.00 B)


Entrenando: SimpleCNN

Parámetros totales: 226,969
Tamaño estimado del modelo: 0.9 MB
Epoch 1/30
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0954 - loss: 3.0357
Epoch 1: val_accuracy improved from -inf to 0.68130, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/SimpleCNN.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 30s 56ms/step - accuracy: 0.0962 - loss: 3.0326 - val_accuracy: 0.6813 - val_loss: 1.3298 - learning_rate: 0.0010
Epoch 2/30
  1/343 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.3594 - loss: 1.7075
Epoch 2: val_accuracy did not improve from 0.68130
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3594 - loss: 1.7075 - val_accuracy: 0.6740 - val_loss: 1.3163 - learning_rate: 0.0010
Epoch 3/30
342/343 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.4638 - loss: 1.6743
Epoch 3: val_accuracy improved from 0.68130 to 0.85686, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/SimpleCNN.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 

### 7.2. ImprovedCNN

In [24]:
# Crear y entrenar ImprovedCNN
model_improved = create_improved_cnn(num_classes=NUM_CLASSES)
model_improved.summary()

history_improved = train_model(
    model_improved,
    'ImprovedCNN',
    X_train, y_train_cat,
    X_val, y_val_cat
)

Model: "ImprovedCNN"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 28, 28, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 14, 14, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 14, 14, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 14, 14, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_3 (Activation)       │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 7, 7, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 7, 7, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_4 (Activation)       │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 3, 3, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼─────────────

 Total params: 442,745 (1.69 MB)

 Trainable params: 441,593 (1.68 MB)

 Non-trainable params: 1,152 (4.50 KB)


Entrenando: ImprovedCNN

Parámetros totales: 442,745
Tamaño estimado del modelo: 1.7 MB
Epoch 1/30
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.2927 - loss: 2.7061
Epoch 1: val_accuracy improved from -inf to 0.22765, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/ImprovedCNN.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 47s 86ms/step - accuracy: 0.2945 - loss: 2.6975 - val_accuracy: 0.2276 - val_loss: 2.5873 - learning_rate: 0.0010
Epoch 2/30
  1/343 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.7656 - loss: 0.7120
Epoch 2: val_accuracy did not improve from 0.22765
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7656 - loss: 0.7120 - val_accuracy: 0.2165 - val_loss: 2.6322 - learning_rate: 0.0010
Epoch 3/30
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8108 - loss: 0.5677
Epoch 3: val_accuracy improved from 0.22765 to 0.98707, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/ImprovedCNN.keras
343/343 ━━━━━━━━━━━━━━

### 7.3. DeepCNN

In [25]:
# Crear y entrenar DeepCNN
model_deep = create_deep_cnn(num_classes=NUM_CLASSES)
model_deep.summary()

history_deep = train_model(
    model_deep,
    'DeepCNN',
    X_train, y_train_cat,
    X_val, y_val_cat
)

Model: "DeepCNN"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 28, 28, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_9 (Conv2D)   │ (None, 28, 28,    │        320 │ input_layer_3[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        128 │ conv2d_9[0][0]    │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_5        │ (None, 28, 28,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_10 (Conv2D)  │ (None, 28, 28,    │      9,248 │ activation_5[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        128 │ conv2d_10[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_6        │ (None, 28, 28,    │          0 │ batch_normalizat… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_11 (Conv2D)  │ (None, 28, 28,    │      9,248 │ activation_6[0][… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 28, 28,    │        128 │ conv2d_11[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 28, 28,    │          0 │ batch_normalizat… │
│                     │ 32)               │            │ activation_5[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 28, 28,    │          0 │ add[0][0]         │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_7     │ (None, 14, 14,    │          0 │ activation_7[0][… │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_10          │ (None, 14, 14,    │          0 │ max_pooling2d_7[… │
│ (Dropout)           │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_12 (Conv2D)  │ (None, 14, 14,    │     18,496 │ dropout_10[0][0]  │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 14, 14,    │        256 │ conv2d_12[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 14, 14,    │          0 │ batch_normalizat… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_13 (Conv2D)  │ (None, 14, 14,    │     36,928 │ activation_8[0][

 Total params: 136,409 (532.85 KB)

 Trainable params: 135,321 (528.60 KB)

 Non-trainable params: 1,088 (4.25 KB)


Entrenando: DeepCNN

Parámetros totales: 136,409
Tamaño estimado del modelo: 0.5 MB
Epoch 1/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.2316 - loss: 2.8033
Epoch 1: val_accuracy improved from -inf to 0.06083, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/DeepCNN.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 30s 55ms/step - accuracy: 0.2322 - loss: 2.8007 - val_accuracy: 0.0608 - val_loss: 6.8447 - learning_rate: 0.0010
Epoch 2/30
  1/343 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.7031 - loss: 0.8688
Epoch 2: val_accuracy improved from 0.06083 to 0.06465, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/DeepCNN.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7031 - loss: 0.8688 - val_accuracy: 0.0647 - val_loss: 6.7419 - learning_rate: 0.0010
Epoch 3/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8142 - loss: 0.5742
Epoch 3: val_accuracy improved from 0.06465 to 0.81679, saving model to /content/driv

In [26]:

# Control de memoria antes de Transfer Learning
if IN_COLAB:
    import psutil
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    mem_used_gb = mem_info.rss / (1024**3)
    
    print(f"\n{'='*60}")
    print(f"ADVERTENCIA: Modelos Transfer Learning requieren MUCHA MEMORIA")
    print(f"{'='*60}")
    print(f"Memoria actual usada: {mem_used_gb:.2f} GB")
    print(f"Memoria disponible: ~{(12 - mem_used_gb):.2f} GB")
    print(f"\nLos modelos Transfer Learning necesitan ~2-3 GB cada uno")
    print(f"Si tienes < 4 GB disponible, SALTAREMOS estos entrenamientos")
    
    if mem_used_gb > 8 and not TRAIN_ALL_MODELS:
        print("MEMORIA INSUFICIENTE: Saltando Transfer Learning...")
        TRAIN_ALL_MODELS = False
    elif TRAIN_ALL_MODELS:
        print("ADVERTENCIA: Intentando entrenar Transfer Learning (riesgo de memoria)")
else:
    print("\nEntorno Local - Sin restricciones de memoria")


ADVERTENCIA: Modelos Transfer Learning requieren MUCHA MEMORIA
Memoria actual usada: 3.13 GB
Memoria disponible: ~8.87 GB

Los modelos Transfer Learning necesitan ~2-3 GB cada uno
Si tienes < 4 GB disponible, SALTAREMOS estos entrenamientos


### 7.4. Transfer Learning - EfficientNetB0

In [27]:
# Crear y entrenar EfficientNetB0
model_efficient = create_transfer_learning_model(
    base_model_name='EfficientNetB0',
    num_classes=NUM_CLASSES,
    trainable=False
)
model_efficient.summary()

history_efficient = train_model(
    model_efficient,
    'TL_EfficientNetB0',
    X_train, y_train_cat,
    X_val, y_val_cat,
    use_augmentation=True
)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "TL_EfficientNetB0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_4 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 28, 28, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing (Resizing)             │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ efficientnetb0 (Functional)     │ (None, 1280)           │     4,049,571 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_13 (Dropout)            │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 25)             │         3,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,216,770 (16.09 MB)

 Trainable params: 167,199 (653.12 KB)

 Non-trainable params: 4,049,571 (15.45 MB)


Entrenando: TL_EfficientNetB0

Parámetros totales: 4,216,770
Tamaño estimado del modelo: 16.1 MB
Epoch 1/30
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step - accuracy: 0.0446 - loss: 3.2246
Epoch 1: val_accuracy improved from -inf to 0.04662, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/TL_EfficientNetB0.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 220s 426ms/step - accuracy: 0.0446 - loss: 3.2245 - val_accuracy: 0.0466 - val_loss: 3.1831 - learning_rate: 0.0010
Epoch 2/30
  1/343 ━━━━━━━━━━━━━━━━━━━━ 4s 14ms/step - accuracy: 0.0781 - loss: 3.1843
Epoch 2: val_accuracy did not improve from 0.04662
343/343 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step - accuracy: 0.0781 - loss: 3.1843 - val_accuracy: 0.0466 - val_loss: 3.1833 - learning_rate: 0.0010
Epoch 3/30
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.0414 - loss: 3.1908
Epoch 3: val_accuracy did not improve from 0.04662
343/343 ━━━━━━━━━━━━━━━━━━━━ 9s 27ms/step - accuracy: 0.0415 - loss: 3.1908 - val_accuracy: 0.0466 - 

### 7.5. Transfer Learning - MobileNetV2

In [28]:
# Crear y entrenar MobileNetV2
model_mobile = create_transfer_learning_model(
    base_model_name='MobileNetV2',
    num_classes=NUM_CLASSES,
    trainable=False
)
model_mobile.summary()

history_mobile = train_model(
    model_mobile,
    'TL_MobileNetV2',
    X_train, y_train_cat,
    X_val, y_val_cat,
    use_augmentation=True
)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "TL_MobileNetV2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_6 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 28, 28, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_1 (Resizing)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 1280)           │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 25)             │         3,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,425,183 (9.25 MB)

 Trainable params: 167,199 (653.12 KB)

 Non-trainable params: 2,257,984 (8.61 MB)


Entrenando: TL_MobileNetV2

Parámetros totales: 2,425,183
Tamaño estimado del modelo: 9.3 MB
Epoch 1/30
342/343 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.1114 - loss: 3.0344
Epoch 1: val_accuracy improved from -inf to 0.33145, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/TL_MobileNetV2.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 144s 284ms/step - accuracy: 0.1117 - loss: 3.0335 - val_accuracy: 0.3315 - val_loss: 2.3803 - learning_rate: 0.0010
Epoch 2/30
  1/343 ━━━━━━━━━━━━━━━━━━━━ 3s 10ms/step - accuracy: 0.1719 - loss: 2.6977
Epoch 2: val_accuracy did not improve from 0.33145
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1719 - loss: 2.6977 - val_accuracy: 0.3294 - val_loss: 2.3794 - learning_rate: 0.0010
Epoch 3/30
342/343 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.2135 - loss: 2.6233
Epoch 3: val_accuracy improved from 0.33145 to 0.40066, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/TL_MobileNetV2.keras
343/343 

### 7.6. Transfer Learning - ResNet50

In [29]:
# Crear y entrenar ResNet50
model_resnet = create_transfer_learning_model(
    base_model_name='ResNet50',
    num_classes=NUM_CLASSES,
    trainable=False
)
model_resnet.summary()

history_resnet = train_model(
    model_resnet,
    'TL_ResNet50',
    X_train, y_train_cat,
    X_val, y_val_cat,
    use_augmentation=True
)

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


Model: "TL_ResNet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 28, 28, 1)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_17 (Conv2D)              │ (None, 28, 28, 3)      │             6 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resizing_2 (Resizing)           │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_18 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 25)             │         3,225 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,853,215 (90.99 MB)

 Trainable params: 265,503 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)


Entrenando: TL_ResNet50

Parámetros totales: 23,853,215
Tamaño estimado del modelo: 91.0 MB
Epoch 1/30
343/343 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0458 - loss: 3.7766
Epoch 1: val_accuracy improved from -inf to 0.04662, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/TL_ResNet50.keras
343/343 ━━━━━━━━━━━━━━━━━━━━ 52s 99ms/step - accuracy: 0.0458 - loss: 3.7754 - val_accuracy: 0.0466 - val_loss: 3.1865 - learning_rate: 0.0010
Epoch 2/30
  1/343 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.0469 - loss: 3.1825
Epoch 2: val_accuracy did not improve from 0.04662
343/343 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0469 - loss: 3.1825 - val_accuracy: 0.0466 - val_loss: 3.1866 - learning_rate: 0.0010
Epoch 3/30
341/343 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.0570 - loss: 3.1738
Epoch 3: val_accuracy improved from 0.04662 to 0.08159, saving model to /content/drive/MyDrive/proyecto-app-lenguaje-senas/models/TL_ResNet50.keras
343/343 ━━━━━━━━━━

## 8. Evaluación en Conjunto de Test

In [33]:
# Lista de modelos a evaluar
if TRAIN_ALL_MODELS or not IN_COLAB:
    # Todos los modelos (máquina local o Colab con suficiente memoria)
    model_names = [
        'SimpleCNN',
        'ImprovedCNN', 
        'DeepCNN',
        'TL_EfficientNetB0',
        'TL_MobileNetV2',
        'TL_ResNet50'
    ]
    print("Evaluando TODOS los modelos...")
else:
    # Solo modelos ligeros para Colab
    model_names = [
        'SimpleCNN',
        'ImprovedCNN', 
        'DeepCNN',
    ]
    print("Evaluando modelos CNN ligeros...")
    print("Para evaluar Transfer Learning, entrena primero esos modelos")

# Evaluar cada modelo
results = []

for model_name in model_names:
    model_path = DIRS['models'] / f"{model_name}.keras"
    
    if not model_path.exists():
        print(f"Modelo no encontrado: {model_name}")
        continue
    
    print(f"\nEvaluando {model_name}...")
    
    # Cargar modelo
    model = keras.models.load_model(str(model_path))
    
    # Evaluar
    test_loss, test_acc = model.evaluate(X_test, y_test_cat, verbose=0)
    
    # Predicciones
    y_pred = model.predict(X_test, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    # Convertir y_test a índices si está en one-hot
    if len(y_test_cat.shape) == 2:
        y_test_classes = np.argmax(y_test_cat, axis=1)
    else:
        y_test_classes = y_test_cat
    
    # Métricas
    precision, recall, f1, _ = precision_recall_fscore_support(
        y_test_classes, y_pred_classes, average='weighted'
    )
    
    results.append({
        'Modelo': model_name,
        'Test Loss': test_loss,
        'Test Accuracy': test_acc,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1
    })
    
    print(f"  Test Accuracy: {test_acc:.4f}")
    print(f"  F1-Score: {f1:.4f}")

# Crear DataFrame de resultados
if results:
    results_df = pd.DataFrame(results)
    results_df = results_df.sort_values('Test Accuracy', ascending=False)
    
    print("RESULTADOS FINALES")
    print(results_df.to_string(index=False))
    
    # Guardar resultados
    results_df.to_csv(DIRS['results_reports'] / 'model_comparison.csv', index=False)
    print(f"\nResultados guardados")
else:
    print("No hay modelos para evaluar")

Evaluando TODOS los modelos...

Evaluando SimpleCNN...
  Test Accuracy: 0.9897
  F1-Score: 0.9897

Evaluando ImprovedCNN...
  Test Accuracy: 0.9916
  F1-Score: 0.9915

Evaluando DeepCNN...
  Test Accuracy: 0.9989
  F1-Score: 0.9989

Evaluando TL_EfficientNetB0...
  Test Accuracy: 0.0201
  F1-Score: 0.0008

Evaluando TL_MobileNetV2...
  Test Accuracy: 0.4260
  F1-Score: 0.4190

Evaluando TL_ResNet50...
  Test Accuracy: 0.1366
  F1-Score: 0.0944
RESULTADOS FINALES
           Modelo  Test Loss  Test Accuracy  Precision   Recall  F1-Score
          DeepCNN   0.011843       0.998885   0.998896 0.998885  0.998885
      ImprovedCNN   0.028768       0.991634   0.992452 0.991634  0.991529
        SimpleCNN   0.050001       0.989682   0.990123 0.989682  0.989674
   TL_MobileNetV2   1.969563       0.425962   0.457098 0.425962  0.418975
      TL_ResNet50   2.760304       0.136643   0.125231 0.136642  0.094360
TL_EfficientNetB0   3.201117       0.020078   0.000403 0.020078  0.000790

Resultados gua

## 9. Visualización de Resultados

In [43]:
# Resumen de resultados
if results:
    # Tabla de resultados con formato
    print("RESULTADOS FINALES - COMPARACIÓN DE MODELOS")
    print(results_df.to_string(index=False))
    
    # Resumen final
    best_model = results_df.iloc[0]
    print(f"\nMEJOR MODELO: {best_model['Modelo']}")
    print(f"   • Accuracy: {best_model['Test Accuracy']:.4f} ({best_model['Test Accuracy']*100:.2f}%)")
    print(f"   • F1-Score: {best_model['F1-Score']:.4f}")
    print(f"   • Precision: {best_model['Precision']:.4f}")
    print(f"   • Recall: {best_model['Recall']:.4f}")

    # Guardar resultados CSV
    results_df.to_csv(DIRS['results_reports'] / 'model_comparison.csv', index=False)
    print(f"\nResultados guardados en: results/reports/model_comparison.csv")
    
else:
    print("No hay resultados para visualizar")

RESULTADOS FINALES - COMPARACIÓN DE MODELOS
           Modelo  Test Loss  Test Accuracy  Precision   Recall  F1-Score
          DeepCNN   0.011843       0.998885   0.998896 0.998885  0.998885
      ImprovedCNN   0.028768       0.991634   0.992452 0.991634  0.991529
        SimpleCNN   0.050001       0.989682   0.990123 0.989682  0.989674
   TL_MobileNetV2   1.969563       0.425962   0.457098 0.425962  0.418975
      TL_ResNet50   2.760304       0.136643   0.125231 0.136642  0.094360
TL_EfficientNetB0   3.201117       0.020078   0.000403 0.020078  0.000790

MEJOR MODELO: DeepCNN
   • Accuracy: 0.9989 (99.89%)
   • F1-Score: 0.9989
   • Precision: 0.9989
   • Recall: 0.9989

Resultados guardados en: results/reports/model_comparison.csv
